In [15]:
from sklearn import preprocessing
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
import pickle
import glob
import pprint
import psycopg2 as pg
import pandas.io.sql as psql
import datetime
import pandas as pd
from sklearn.metrics import f1_score

# load comments with calculated attributes

In [30]:
#save complete submissions
path = 'D:\\skola\\ING\\0.DP\\0.dataset\\6. Reddit Russian trolls\\0.data\\'
filename = 'attributes.pkl'
with open(path + filename, 'rb') as f:
    dataFrameComments = pickle.load(f)

# load users

In [11]:
#suspicious users
text_file = open('D:\\skola\\ING\\0.DP\\0.dataset\\6. Reddit Russian trolls\\suspiciousAccounts.txt', "r")
suspiciousUsers = text_file.read().split(',')
text_file.close()

#1000 regular users
path = 'D:\\skola\\ING\\0.DP\\0.dataset\\6. Reddit Russian trolls\\0.data\\'
filename = 'regularUsers_1000.pkl'
with open(path + filename, 'rb') as f:
    regularUsers = pickle.load(f)

#blacklisted regular users
path = 'D:\\skola\\ING\\0.DP\\0.dataset\\6. Reddit Russian trolls\\0.data\\'
filename = 'blacklistedRegularUsers.pkl'
with open(path + filename, 'rb') as f:
    blacklistedRegularUsers = pickle.load(f)

#remove blacklisted users
for user in blacklistedRegularUsers:
    regularUsers.remove(user)


In [15]:
# len(regularUsers) = 239
# len(suspiciousUsers) = 944

# for every user:
## load comments, calculate attributes by weeks -> save to dataframe

In [29]:
# connect to the database
try:
    conn = pg.connect("dbname='DP' user='postgres' host='localhost' password='root'")
except:
    print ("Unable to connect to the database")

In [218]:
#create new empty dataframe
columns = ['author', 'suspicious']
dataFrameUsers = pd.DataFrame(columns=columns)


dfLineIndex = 0

for user in suspiciousUsers + regularUsers:
    # get user comment ids
    comments = psql.read_sql("SELECT * FROM comments WHERE author = '" + user + "'", conn)
    
    # sort comments from oldest to newest 
    sortedComments = comments.sort_values(by='created_utc', ascending=True)
    
    startDT = 0
    endDT = 0
    days = 7
    weekIndex = 0

    for index, row in sortedComments.iterrows():
        # use that id to find calculated attributes for particular comment
        commentAttributes = dataFrameComments.loc[dataFrameComments['comment_id'] == row['id']]
        if(len(commentAttributes) == 0):
            print('something went wrong, skipping...')
            continue
        currentDT = datetime.datetime.fromtimestamp(row['created_utc'])

        #append author name and label
        dataFrameUsers.at[dfLineIndex, 'author'] = row['author']
        if (user in suspiciousUsers):
            dataFrameUsers.at[dfLineIndex, 'suspicious'] = 1
        else:
            dataFrameUsers.at[dfLineIndex, 'suspicious'] = 0
            
        if startDT == 0:
            startDT = currentDT
            endDT = startDT + datetime.timedelta(days=days)

            #reset sum of attributes
            numof_older_comments = 0
            numof_parent_comments = 0
            numof_replies = 0
            numof_authors = 0
            numof_author_older_comments = 0

        while (currentDT > endDT):
            weekIndex += 1
            startDT = endDT
            endDT = startDT + datetime.timedelta(days=days)
            #print('empty week', weekIndex)

            #reset sum of attributes
            numof_older_comments = 0
            numof_parent_comments = 0
            numof_replies = 0
            numof_authors = 0
            numof_author_older_comments = 0

        # sum calculated attributes every 7 days starting at the datetime of first comment 
        numof_older_comments += commentAttributes['numof_older_comments'].values[0]
        numof_parent_comments += commentAttributes['numof_parent_comments'].values[0]
        numof_replies += commentAttributes['numof_replies'].values[0]
        numof_authors += commentAttributes['numof_authors'].values[0]
        numof_author_older_comments += commentAttributes['numof_author_older_comments'].values[0]

        #append sum attributes
        dataFrameUsers.at[dfLineIndex, 'week_' + str(weekIndex) + '_numof_older_comments'] = numof_older_comments
        dataFrameUsers.at[dfLineIndex, 'week_' + str(weekIndex) + '_numof_parent_comments'] = numof_parent_comments
        dataFrameUsers.at[dfLineIndex, 'week_' + str(weekIndex) + '_numof_replies'] = numof_replies
        dataFrameUsers.at[dfLineIndex, 'week_' + str(weekIndex) + '_numof_authors'] = numof_authors
        dataFrameUsers.at[dfLineIndex, 'week_' + str(weekIndex) + '_numof_author_older_comments'] = numof_author_older_comments

        #print("week", numof_older_comments)
        #print("utc", row['created_utc'])

    dfLineIndex += 1
    

something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went

something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went

something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went wrong, skipping...
something went

#  backup user attributes

In [220]:
#save complete submissions
path = 'D:\\skola\\ING\\0.DP\\0.dataset\\6. Reddit Russian trolls\\0.data\\'
filename = 'user_attributes.pkl'
with open(path + filename, 'wb') as f:
    pickle.dump(dataFrameUsers, f)

In [ ]:
#only 116 suspicious users because many users made no comments at all and only created submissions
dataFrameUsers

In [24]:
dataFrameUsersBackup = dataFrameUsers

In [25]:
dataFrameUsers = dataFrameUsers.fillna(0)

In [26]:
X_scaled = preprocessing.scale(dataFrameUsers.iloc[:,2:1487])
X = X_scaled
y = dataFrameUsers['suspicious'].values.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:

#X = X_scaled
#y = dataFrame['suspicious'].values.tolist()

clf = svm.SVC(gamma=0.1)
clf.fit(X_train, y_train)  
#SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
#    max_iter=-1, probability=False, random_state=None, shrinking=True,
#    tol=0.001, verbose=False)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [28]:
tst = clf.predict(X_test)

In [29]:
i = 0
trafilSuspicious = 0
trafilRegular = 0
netrafilSuspicious = 0
netrafilRegular = 0

for x in tst:
    if (x == y_test[i]):
        if (x == 1):
            trafilSuspicious += 1
        else:
            trafilRegular += 1
    else:
        if (x == 1):
            netrafilRegular += 1
        else:
            netrafilSuspicious += 1
    i += 1
print("trafilSuspicious: ", trafilSuspicious / (trafilSuspicious + netrafilSuspicious))
print("trafilRegular: ", trafilRegular / (trafilRegular + netrafilRegular))
print("netrafilSuspicious: ", netrafilSuspicious)
print("netrafilRegular: ", netrafilRegular)


trafilSuspicious:  0.9534883720930233
trafilRegular:  0.7837837837837838
netrafilSuspicious:  2
netrafilRegular:  16


In [30]:
f1_score(y_test, tst, average=None) 

array([0.86567164, 0.82      ])

In [31]:
from sklearn.metrics import confusion_matrix
y_true = y_test
y_pred = tst
confusion_matrix(y_true, y_pred)


array([[58, 16],
       [ 2, 41]], dtype=int64)

In [37]:
from sklearn.metrics import precision_recall_fscore_support
y_true = y_test
y_pred = tst
results = precision_recall_fscore_support(y_true, y_pred, average=None)
print("precision: \t", results[0])
print("recall: \t", results[1])
print("f1: \t\t", results[2])
print("support: \t", results[3])


precision: 	 [0.96666667 0.71929825]
recall: 	 [0.78378378 0.95348837]
f1: 		 [0.86567164 0.82      ]
support: 	 [74 43]


In [239]:
len(y_train) + len(y_test)

353

In [242]:
sum(y_train) + sum(y_test)

116

In [ ]:
#after removal of invalid data
# len(regularUsers) = 237
# len(suspiciousUsers) = 116

In [232]:
y_test

[0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0]

In [13]:
#example without stadardization
dataFrameUsersBackup = dataFrameUsers
dataFrameUsers = dataFrameUsers.fillna(0)

X_scaled = preprocessing.scale(dataFrameUsers.iloc[:,2:1487])
X = dataFrameUsers.iloc[:,2:1487]
y = dataFrameUsers['suspicious'].values.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


#X = X_scaled
#y = dataFrame['suspicious'].values.tolist()

clf = svm.SVC(gamma=0.1)
clf.fit(X_train, y_train)  
#SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
#    max_iter=-1, probability=False, random_state=None, shrinking=True,
#    tol=0.001, verbose=False)

tst = clf.predict(X_test)

i = 0
trafilSuspicious = 0
trafilRegular = 0
netrafilSuspicious = 0
netrafilRegular = 0

for x in tst:
    if (x == y_test[i]):
        if (x == 1):
            trafilSuspicious += 1
        else:
            trafilRegular += 1
    else:
        if (x == 1):
            netrafilRegular += 1
        else:
            netrafilSuspicious += 1
    i += 1
print("trafilSuspicious: ", trafilSuspicious / (trafilSuspicious + netrafilSuspicious))
print("trafilRegular: ", trafilRegular / (trafilRegular + netrafilRegular))
print("netrafilSuspicious: ", netrafilSuspicious)
print("netrafilRegular: ", netrafilRegular)


trafilSuspicious:  0.18604651162790697
trafilRegular:  1.0
netrafilSuspicious:  35
netrafilRegular:  0
